In [1]:
#### download model and tokenizer

!wget https://huggingface.co/lingxusb/PlasmidGPT/resolve/main/pretrained_model.pt
!wget https://huggingface.co/lingxusb/PlasmidGPT/resolve/main/addgene_trained_dna_tokenizer.json

--2024-10-05 07:45:15--  https://huggingface.co/lingxusb/PlasmidGPT/resolve/main/pretrained_model.pt
Resolving huggingface.co (huggingface.co)... 3.164.110.77, 3.164.110.128, 3.164.110.114, ...
Connecting to huggingface.co (huggingface.co)|3.164.110.77|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/39/1b/391b3344dcaf5b835fcc8bdaa650824a769edb190566148b45b4de13176e8be8/12c95688b0244c88b2a660c18214f15f76896c1b935cc51ef97f073b776f9ad3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27pretrained_model.pt%3B+filename%3D%22pretrained_model.pt%22%3B&Expires=1728373515&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyODM3MzUxNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzM5LzFiLzM5MWIzMzQ0ZGNhZjViODM1ZmNjOGJkYWE2NTA4MjRhNzY5ZWRiMTkwNTY2MTQ4YjQ1YjRkZTEzMTc2ZThiZTgvMTJjOTU2ODhiMDI0NGM4OGIyYTY2MGMxODIxNGYxNWY3Njg5NmMxYjkzNWNjNTFlZjk3ZjA3M2I3NzZmOWFkMz9yZXNwb25z

In [4]:
#### generate sequence

import torch
from transformers import PreTrainedTokenizerFast, GenerationConfig

# Define the start sequence and sequence generation number
sequence_number = 1
start_sequence = (
    'CCAATTATTGAAGGCCTCCCTAACGGGGGGCCTTTTTTTGTTTCTGGTCTCCCgcttGATAAGTCCCTAACTTTTACAGCTAGCTCAGTCCTAGGTATTATGCTAGCCTGAAGCTGTCACCGGATGTGCTTTCCGGTCTGATGAGTCCGTGAGGACGAAACAGCCTCTACAAATAATTTTGTTTAATACTAGAGAAAGAGGGGAAATACTAGATGGTGAGCAAGGGCGAGGAGCTGTTCACCGGGGTGGTGCCCATCCTGGTCGAGCTGGACGGCGACGTAAACGGCCACAAGTTCAGCGTGTCCGGCGAGGGCGAGGGCGATGCCACCTACGGCAAGCTGACCCTGAAGTTCATCTGCACCACAGGCAAGCTGCCCGTGCCCTGGCCCACCCTCGTGACCACCTTCGGCTACGGCCTGCAATGCTTCGCCCGCTACCCCGACCACATGAAGCTGCACGACTTCTTCAAGTCCGCCATGCCCGAAGGCTACGTCCAGGAGCGCACCATCTTCTTCAAGGACGACGGCAACTACAAGACCCGCGCCGAGGTGAAGTTCGAGGGCGACACCCTGGTGAACCGCATCGAGCTGAAGGGCATCGACTTCAAGGAGGACGGCAACATCCTGGGGCACAAGCTGGAGTACAACTACAACAGCCACAACGTCTATATCATGGCCGACAAGCAGAAGAACGGCATCAAGGTGAACTTCAAGATCCGCCACAACATCGAGGACGGCAGCGTGCAGCTCGCCGACCACTACCAGCAGAACACCCCAATCGGCGACGGCCCCGTGCTGCTGCCCGACAACCACTACCTTAGCTACCAGTCCGCCCTGAGCAAAGACCCCAACGAGAAGCGCGATCACATGGTCCTGCTGGAGTTCGTGACCGCCGCCGGGATCACTCTCGGCATGGACGAGCTGTACAAGTAA'
)

# Load the pre-trained model and tokenizer
pt_file_path = 'pretrained_model.pt'
model = torch.load(pt_file_path)
model.eval()

tokenizer = PreTrainedTokenizerFast(tokenizer_file='addgene_trained_dna_tokenizer.json')
special_tokens_dict = {'additional_special_tokens': ['[PROMPT]', '[PROMPT2]']}
tokenizer.add_special_tokens(special_tokens_dict)

# Set the device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("GPU not available, using CPU.")

# Print information about the start sequence
print("Default start sequence is the YFP expression cassette.")
print(f"Start sequence: {start_sequence}")

# Tokenize and prepare input
input_ids = tokenizer.encode(start_sequence.upper(), return_tensors='pt').to(device)

# Add special tokens to the input
special_tokens = torch.tensor([3] * 10 + [2], dtype=torch.long, device=device)
input_ids = torch.cat((special_tokens.unsqueeze(0), input_ids), dim=1)

# Sequence generation loop
for j in range(sequence_number):
    while True:
        # Generate sequences with the model
        outputs = model.generate(
            input_ids,
            max_length=300,
            num_return_sequences=1,
            temperature=1.0,
            do_sample=True,
            generation_config=GenerationConfig.from_model_config(model.config)
        )

        # Decode and clean the generated sequence
        generated_sequence = tokenizer.decode(outputs[0], skip_special_tokens=True).replace(" ", "")
        if len(generated_sequence) > 200 + len(start_sequence):
            break

    print(f"Generated sequence: {generated_sequence}")

    # Write the generated sequence to a FASTA file
    fasta_file_name = "generated_sequence.fasta"
    with open(fasta_file_name, 'w') as file:
        file.write(f">PlasmidGPT_generate{j}\n")
        file.write(generated_sequence + "\n")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GPU not available, using CPU.
Default start sequence is the YFP expression cassette.
Start sequence: CCAATTATTGAAGGCCTCCCTAACGGGGGGCCTTTTTTTGTTTCTGGTCTCCCgcttGATAAGTCCCTAACTTTTACAGCTAGCTCAGTCCTAGGTATTATGCTAGCCTGAAGCTGTCACCGGATGTGCTTTCCGGTCTGATGAGTCCGTGAGGACGAAACAGCCTCTACAAATAATTTTGTTTAATACTAGAGAAAGAGGGGAAATACTAGATGGTGAGCAAGGGCGAGGAGCTGTTCACCGGGGTGGTGCCCATCCTGGTCGAGCTGGACGGCGACGTAAACGGCCACAAGTTCAGCGTGTCCGGCGAGGGCGAGGGCGATGCCACCTACGGCAAGCTGACCCTGAAGTTCATCTGCACCACAGGCAAGCTGCCCGTGCCCTGGCCCACCCTCGTGACCACCTTCGGCTACGGCCTGCAATGCTTCGCCCGCTACCCCGACCACATGAAGCTGCACGACTTCTTCAAGTCCGCCATGCCCGAAGGCTACGTCCAGGAGCGCACCATCTTCTTCAAGGACGACGGCAACTACAAGACCCGCGCCGAGGTGAAGTTCGAGGGCGACACCCTGGTGAACCGCATCGAGCTGAAGGGCATCGACTTCAAGGAGGACGGCAACATCCTGGGGCACAAGCTGGAGTACAACTACAACAGCCACAACGTCTATATCATGGCCGACAAGCAGAAGAACGGCATCAAGGTGAACTTCAAGATCCGCCACAACATCGAGGACGGCAGCGTGCAGCTCGCCGACCACTACCAGCAGAACACCCCAATCGGCGACGGCCCCGTGCTGCTGCCCGACAACCACTACCTTAGCTACCAGTCCGCCCTGAGCAAAGACCCCAACGAGAAGCGCGATCACATGGTCCTGCTGGAGTTCGTGACCGCCGCCGGG